In [1]:
#%pip install quantstats

In [ ]:
import os
import sys
import glob
import json
import numpy as np
import pandas as pd
import warnings
import traceback
from typing import Optional, Dict
import quantstats as qs 

# --- Environment Setup ---
os.environ['LD_LIBRARY_PATH'] = f"{os.environ.get('HOME')}/kohv04/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"
sys.path.append('/home/jupyter-kohv04@vse.cz/kohv04/lib/python3.10/site-packages')

try:
    import talib
except ImportError as e:
    print(f"Failed to import TA-Lib: {e}")
import vectorbt as vbt
from numba import njit
from vectorbt.portfolio.enums import SizeType

# -global settings
vbt.settings.caching['enabled'] = False
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning, module="pandas.core.frame")

# config and constrants
BASE_DIR = "/home/jupyter-kohv04@vse.cz/kohv04/backtesting_final/"
METADATA_FILE = f"{BASE_DIR}/metadata/nasdaq100_ticker_dataset.json"
INITIAL_CASH = 1_000_000
REJECT_PROBABILITY = 0.005 # 0.5% chance of order rejection

# Best Parameter Grids for Simulation
BEST_BASELINE_BREAKOUT_PARAMS = {'sl_stop': [0.011], 'tp_stop': [0.01]}
BEST_BASELINE_BBANDS_PARAMS = {'timeperiod': [25], 'nbdev': [2.5]}
BEST_BASELINE_MOMENTUM_PARAMS = {'window': [6], 'sl_stop': [0.024]}
BEST_VOLUME_MOMENTUM_PARAMS = {'timeperiod': [17], 'kappa_vol_mom': [2.8], 'adx_threshold': [33], 'alpha_atr': [3.3]}
BEST_VOLUME_BREAKOUT_PARAMS = {'phi_va': [0.80], 'kappa_surge': [3.5], 'timeperiod': [20], 'adx_threshold': [20], 'alpha_atr': [3.0], 'alpha_tp': [6.0]}
BEST_VOLUME_VWAP_REVERSION_PARAMS = {'window': [40], 'quantile': [0.9], 'slope': [0.0001], 'tau_vwap_trend': [19], 'alpha_atr': [3.2], 'alpha_tp': [6.0]}
BEST_DL_BREAKOUT_PARAMS = {'phi_va': [0.75], 'kappa_dl': [1.6], 'timeperiod': [16], 'adx_threshold': [20], 'alpha_atr': [3.0], 'alpha_tp': [4.0]}
BEST_DL_VOLUME_MOMENTUM_PARAMS = {'timeperiod': [20], 'kappa_dl': [1.5], 'adx_threshold': [30], 'alpha_atr': [4.0], 'tau_vol_trend': [7]}
BEST_DL_VWAP_REVERSION_PARAMS = {'delta_vwap': [0.003], 'tau_vwap_trend': [18], 'volume_multiplier': [1.5], 'alpha_atr': [3.0], 'alpha_tp': [7.0]}


In [ ]:
def load_simulation_data(ticker: str, strategy_type: str) -> Optional[pd.DataFrame]:
    """Loads and combines parquet files for a given ticker and strategy type for simulation."""
    path = os.path.join(BASE_DIR, f"ticker={ticker}_standardized", f"simulation_indicators_{strategy_type}")
    all_files = glob.glob(os.path.join(path, "part.*.parquet"))
    if not all_files:
        print(f"Warning: No simulation files found for {ticker} in {path}")
        return None
    try:
        df = pd.concat((pd.read_parquet(f) for f in all_files), ignore_index=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values('timestamp').set_index('timestamp')
        return df
    except Exception as e:
        print(f"Error loading simulation data for {ticker} - {strategy_type}: {e}")
        return None

def run_baseline_breakout(df: pd.DataFrame, sl_stop: list, tp_stop: list, **kwargs):
    df.index = pd.to_datetime(df.index)
    param_index = pd.MultiIndex.from_arrays([sl_stop, tp_stop], names=['sl_stop', 'tp_stop'])
    broadcast_target = pd.DataFrame(index=df.index, columns=param_index)
    price_df = df['close'].vbt.broadcast_to(broadcast_target)
    time_mask = (df.index.hour < 15)
    time_filter = pd.Series(time_mask, index=df.index).vbt.broadcast_to(price_df)
    long_entries = df['breakout_buy'].vbt.broadcast_to(broadcast_target).fillna(False) & time_filter
    short_entries = df['breakout_sell'].vbt.broadcast_to(broadcast_target).fillna(False) & time_filter
    sl_stop_params = broadcast_target.columns.get_level_values('sl_stop').to_numpy()
    tp_stop_params = broadcast_target.columns.get_level_values('tp_stop').to_numpy()
    return vbt.Portfolio.from_signals(
        price_df, entries=long_entries, short_entries=short_entries, sl_stop=sl_stop_params,
        tp_stop=tp_stop_params, freq='1min', init_cash=INITIAL_CASH, fees=0.0001,
        slippage=0.0002, size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_baseline_bbands(df: pd.DataFrame, timeperiod: list, nbdev: list, **kwargs):
    df.index = pd.to_datetime(df.index)
    bbands = vbt.talib('BBANDS').run(df['close'], timeperiod=timeperiod[0], nbdevup=nbdev[0], nbdevdn=nbdev[0])
    price_df = df['close'].vbt.broadcast_to(bbands.lowerband)
    time_mask = (df.index.hour < 15)
    time_filter = pd.Series(time_mask, index=df.index).vbt.broadcast_to(price_df)
    long_entries = (price_df < bbands.lowerband) & time_filter
    long_exits = price_df > bbands.middleband
    short_entries = (price_df > bbands.upperband) & time_filter
    short_exits = price_df < bbands.middleband
    return vbt.Portfolio.from_signals(
        price_df, entries=long_entries, exits=long_exits, short_entries=short_entries,
        short_exits=short_exits, freq='1min', init_cash=INITIAL_CASH, fees=0.0001,
        slippage=0.0002, size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_baseline_momentum(df: pd.DataFrame, window: list, sl_stop: list, **kwargs):
    df.index = pd.to_datetime(df.index)
    is_up = (df['close'] > df['open']).astype(int)
    is_down = (df['close'] < df['open']).astype(int)
    consecutive_up = is_up.rolling(window=window[0]).sum()
    long_entries = (consecutive_up >= window[0])
    consecutive_down = is_down.rolling(window=window[0]).sum()
    short_entries = (consecutive_down >= window[0])
    time_mask = (df.index.hour < 15)
    final_long_entries = long_entries & time_mask
    final_short_entries = short_entries & time_mask
    return vbt.Portfolio.from_signals(
        df['close'], entries=final_long_entries, short_entries=final_short_entries,
        sl_stop=sl_stop[0], sl_trail=True, freq='1min', init_cash=INITIAL_CASH, fees=0.0001,
        slippage=0.0002, size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

@njit
def _expanding_profile_nb(a: np.ndarray, phi_va: float, min_periods: int) -> np.ndarray:
    output = np.full((a.shape[0], 3), np.nan, dtype=np.float64)
    close_col, volume_col = a[:, 0], a[:, 1]
    for i in range(a.shape[0]):
        if i + 1 >= min_periods:
            current_close_window, current_volume_window = close_col[:i+1], volume_col[:i+1]
            poc, vah, val = np.nan, np.nan, np.nan
            valid_mask = ~np.isnan(current_close_window) & ~np.isnan(current_volume_window)
            close, volume = current_close_window[valid_mask], current_volume_window[valid_mask]
            if close.shape[0] > 0:
                price_bins = np.floor(close * 100) / 100
                unique_prices = np.unique(price_bins)
                daily_profile = np.zeros(len(unique_prices), dtype=np.float64)
                for k in range(len(price_bins)):
                    idx = np.searchsorted(unique_prices, price_bins[k])
                    if idx < len(unique_prices) and unique_prices[idx] == price_bins[k]:
                        daily_profile[idx] += volume[k]
                total_volume = np.sum(daily_profile)
                if total_volume > 0:
                    max_volume_indices = np.where(daily_profile == np.max(daily_profile))[0]
                    poc_idx = max_volume_indices[np.argmin(np.abs(unique_prices[max_volume_indices] - (np.max(unique_prices) + np.min(unique_prices)) / 2))]
                    poc = unique_prices[poc_idx]
                    target_volume = total_volume * phi_va
                    va_indices, va_volume = [poc_idx], daily_profile[poc_idx]
                    up_idx, down_idx = poc_idx + 1, poc_idx - 1
                    while va_volume < target_volume and (up_idx < len(daily_profile) or down_idx >= 0):
                        vol_above = daily_profile[up_idx] if up_idx < len(daily_profile) else -1.0
                        vol_below = daily_profile[down_idx] if down_idx >= 0 else -1.0
                        if vol_above > vol_below:
                            va_indices.append(up_idx)
                            va_volume += vol_above
                            up_idx += 1
                        else:
                            va_indices.append(down_idx)
                            va_volume += vol_below
                            down_idx -= 1
                    value_area_prices = unique_prices[np.array(va_indices, dtype=np.int64)]
                    vah, val = np.max(value_area_prices), np.min(value_area_prices)
            output[i, 0], output[i, 1], output[i, 2] = poc, vah, val
    return output

def calculate_developing_profiles(df: pd.DataFrame, phi_va: float, min_periods: int = 60) -> pd.DataFrame:
    def _apply_kernel_to_day(day_df: pd.DataFrame, phi_va: float, min_p: int) -> pd.DataFrame:
        metrics = _expanding_profile_nb(day_df[['close', 'volume']].values, phi_va, min_p)
        return pd.DataFrame(metrics, index=day_df.index, columns=['poc', 'vah', 'val'])
    daily_results = df.groupby(df.index.normalize()).apply(lambda x: _apply_kernel_to_day(x, phi_va, min_periods))
    daily_results.index = daily_results.index.droplevel(0)
    return daily_results

def run_volume_breakout(df: pd.DataFrame, phi_va: list, kappa_surge: list, timeperiod: list, adx_threshold: list, alpha_atr: list, alpha_tp: list, **kwargs):
    vp = calculate_developing_profiles(df, phi_va[0])
    vah, val = vp['vah'], vp['val']
    adx = vbt.talib('ADX').run(df['high'], df['low'], df['close'], timeperiod=timeperiod[0]).real
    volume_confirm = (df['volume'] > kappa_surge[0] * df['volume_avg'])
    adx_confirm = (adx > adx_threshold[0])
    long_entries = (df['close'] > vah) & volume_confirm & adx_confirm
    short_entries = (df['close'] < val) & volume_confirm & adx_confirm
    time_mask = (df.index.hour < 15)
    final_long_entries, final_short_entries = long_entries & time_mask, short_entries & time_mask
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    tp_pct = (alpha_tp[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=final_long_entries, short_entries=final_short_entries,
        sl_stop=sl_pct, tp_stop=tp_pct, freq='1min', init_cash=INITIAL_CASH, fees=0.0001,
        slippage=0.0002, size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_volume_momentum(df: pd.DataFrame, timeperiod: list, kappa_vol_mom: list, adx_threshold: list, alpha_atr: list, **kwargs):
    adx = vbt.talib('ADX').run(df['high'], df['low'], df['close'], timeperiod=timeperiod[0]).real
    adx_confirm = adx > adx_threshold[0]
    rel_vol_confirm = df['relative_volume'] > kappa_vol_mom[0]
    long_entries = (df['close'] > df['prev_session_high']) & rel_vol_confirm & adx_confirm
    short_entries = (df['close'] < df['prev_session_low']) & rel_vol_confirm & adx_confirm
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=long_entries, short_entries=short_entries, sl_stop=sl_pct,
        sl_trail=True, freq='1min', init_cash=INITIAL_CASH, fees=0.0001, slippage=0.0002,
        size=0.1, size_type=SizeType.Percent, upon_long_conflict='ignore',
        upon_opposite_entry='ignore', reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_volume_vwap_reversion(df: pd.DataFrame, window: list, quantile: list, slope: list, tau_vwap_trend: list, alpha_atr: list, alpha_tp: list, **kwargs):
    vwap_trend = df['vwap_actual'].shift(tau_vwap_trend[0])
    obv_lower_q = df['obv'].rolling(window=window[0]).quantile(quantile[0])
    obv_upper_q = df['obv'].rolling(window=window[0]).quantile(1.0 - quantile[0])
    trend_down, trend_up = df['vwap_actual'] < vwap_trend, df['vwap_actual'] > vwap_trend
    long_entries = (df['close'] < df['vwap_actual']) & (df['obv'] < obv_lower_q) & (df['vwap_actual_slope'] >= slope[0]) & trend_down
    short_entries = (df['close'] > df['vwap_actual']) & (df['obv'] > obv_upper_q) & (df['vwap_actual_slope'] <= -slope[0]) & trend_up
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    tp_pct = (alpha_tp[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=long_entries, short_entries=short_entries, sl_stop=sl_pct,
        tp_stop=tp_pct, freq='1min', init_cash=INITIAL_CASH, fees=0.0001, slippage=0.0002,
        size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_dl_breakout(df: pd.DataFrame, phi_va: list, kappa_dl: list, timeperiod: list, adx_threshold: list, alpha_atr: list, alpha_tp: list, **kwargs):
    profile_df = df[['close']].copy()
    profile_df['volume'] = df['pred_volume_15_tft']
    vp = calculate_developing_profiles(profile_df, phi_va[0])
    vah, val = vp['vah'], vp['val']
    adx = vbt.talib('ADX').run(df['high'], df['low'], df['close'], timeperiod=timeperiod[0]).real
    volume_confirm = (df['pred_volume_15_tft'] > kappa_dl[0] * df['volume_avg'])
    adx_confirm = (adx > adx_threshold[0])
    long_entries = (df['close'] > vah) & volume_confirm & adx_confirm
    short_entries = (df['close'] < val) & volume_confirm & adx_confirm
    time_mask = (df.index.hour < 15)
    final_long_entries, final_short_entries = long_entries & time_mask, short_entries & time_mask
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    tp_pct = (alpha_tp[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=final_long_entries, short_entries=final_short_entries,
        sl_stop=sl_pct, tp_stop=tp_pct, freq='1min', init_cash=INITIAL_CASH, fees=0.0001,
        slippage=0.0002, size=0.1, size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_dl_volume_momentum(df: pd.DataFrame, timeperiod: list, kappa_dl: list, adx_threshold: list, alpha_atr: list, tau_vol_trend: list, **kwargs):
    adx = vbt.talib('ADX').run(df['high'], df['low'], df['close'], timeperiod=timeperiod[0]).real
    vol_trend = df['pred_volume_tft_15_scaled'].rolling(window=tau_vol_trend[0]).mean()
    adx_confirm = adx > adx_threshold[0]
    volume_confirm = vol_trend > kappa_dl[0] * df['volume_avg']
    long_entries = (df['close'] > df['prev_session_high']) & volume_confirm & adx_confirm
    short_entries = (df['close'] < df['prev_session_low']) & volume_confirm & adx_confirm
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=long_entries, short_entries=short_entries, sl_stop=sl_pct,
        sl_trail=True, freq='1min', init_cash=INITIAL_CASH, fees=0.0001, slippage=0.0002,
        size=0.1, size_type=SizeType.Percent, upon_long_conflict='ignore',
        upon_opposite_entry='ignore', reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_dl_vwap_reversion(df: pd.DataFrame, delta_vwap: list, tau_vwap_trend: list, volume_multiplier: list, alpha_atr: list, alpha_tp: list, **kwargs):
    vwap_trend = df['vwap_dl'].shift(tau_vwap_trend[0])
    trend_down, trend_up = df['vwap_dl'] < vwap_trend, df['vwap_dl'] > vwap_trend
    pred_vol_high = df['pred_volume_tft_15_scaled'] > (volume_multiplier[0] * df['volume'])
    price_below_vwap = df['close'] < (1 - delta_vwap[0]) * df['vwap_dl']
    price_above_vwap = df['close'] > (1 + delta_vwap[0]) * df['vwap_dl']
    long_entries = price_below_vwap & trend_down & pred_vol_high
    short_entries = price_above_vwap & trend_up & pred_vol_high
    long_exits = df['close'].vbt.crossed_above(df['vwap_dl'])
    short_exits = df['close'].vbt.crossed_below(df['vwap_dl'])
    sl_pct = (alpha_atr[0] * df['atr']) / df['close']
    tp_pct = (alpha_tp[0] * df['atr']) / df['close']
    return vbt.Portfolio.from_signals(
        df['close'], entries=long_entries, exits=long_exits, short_entries=short_entries,
        short_exits=short_exits, sl_stop=sl_pct, tp_stop=tp_pct, freq='1min',
        init_cash=INITIAL_CASH, fees=0.0001, slippage=0.0002, size=0.1,
        size_type=SizeType.Percent, upon_opposite_entry='ignore',
        reject_prob=REJECT_PROBABILITY, allow_partial=True, **kwargs
    )

def run_and_evaluate_strategy(strategy_name: str, ticker: str, strategy_func, df: pd.DataFrame, params: Dict) -> Optional[pd.Series]:
    """
    Runs a single strategy simulation, prints a report, and saves all relevant
    trade, order, position, log, and time-series data to CSV files.
    """
    print(f"\n{'='*80}")
    print(f"--- Running Simulation for: {strategy_name} on Ticker: {ticker} ---")
    print(f"{'='*80}")

    if df is None or df.empty:
        print(f"Skipping '{strategy_name}' for {ticker} due to missing data.")
        return None

    try:
        single_value_params = {k: v[0] for k, v in params.items()}

        print("DEBUG: Running strategy function...")
        pf = strategy_func(df, **params, log=True)
        print(f"DEBUG: Portfolio object created. Type: {type(pf)}, Shape: {pf.wrapper.shape}")

  
        if pf.wrapper.ndim > 1 and pf.wrapper.shape[1] > 1:
            print(f"DEBUG: Portfolio has multiple columns ({pf.wrapper.shape[1]}). Selecting first column for stats and data saving.")
            pf_single_col = pf.iloc[:, 0]
        else:
            print("DEBUG: Portfolio has a single column.")
            pf_single_col = pf

        print("\n--- Strategy Parameters ---")
        print(pd.Series(single_value_params))

        print("\n--- Performance Summary ---")
        all_metrics = list(pf_single_col.metrics.items())

        try:
            import quantstats as qs
            kelly_metric = ('Kelly Criterion', dict(
                title='Kelly Criterion',
                calc_func=lambda returns: qs.stats.kelly_criterion(returns), 
                tags='risk'
            ))
            all_metrics.append(kelly_metric)
        except ImportError:
            print("Warning: `quantstats` not installed. Skipping Kelly Criterion calculation.")


        print("DEBUG: Calculating stats...")
        stats = pf_single_col.stats(metrics=all_metrics)
        print("DEBUG: Stats calculation complete.")
        print(stats)

        # Saving all simulation data
        print("\n--- Saving All Simulation Data ---")

        results_dir = os.path.join(BASE_DIR, "simulation_results")
        ticker_dir = os.path.join(results_dir, ticker)
        strategy_dir = os.path.join(ticker_dir, strategy_name.replace(' ', '_'))
        os.makedirs(strategy_dir, exist_ok=True)

        def save_data(data_to_save, data_name, s_dir):
            """Helper function to save DataFrame or Series to a CSV file."""
            if data_to_save is not None and not data_to_save.empty:
                filename = os.path.join(s_dir, f"{data_name}.csv")
                index_to_save = isinstance(data_to_save.index, pd.DatetimeIndex)
                if isinstance(data_to_save, pd.Series):
                    data_to_save.to_csv(filename, index=True, header=True)
                else:
                    data_to_save.to_csv(filename, index=index_to_save)
                print(f"    - Saved {data_name} data to {filename}")
            else:
                print(f"    - No data to save for {data_name}")

        print("DEBUG: Saving orders, trades, and positions...")
        save_data(pf_single_col.orders.records_readable, 'orders', strategy_dir)
        save_data(pf_single_col.trades.records_readable, 'trades', strategy_dir)
        save_data(pf_single_col.positions.records_readable, 'positions', strategy_dir)

        print("DEBUG: Checking and saving logs...")
        if pf_single_col.logs is not None and not pf_single_col.logs.records_readable.empty:
            save_data(pf_single_col.logs.records_readable, 'logs', strategy_dir)
        else:
            print("    - No data to save for logs")

        print("DEBUG: Creating and saving timeseries data...")
        timeseries_df = pd.DataFrame({
            'portfolio_value': pf_single_col.value().squeeze(),
            'asset_value': pf_single_col.asset_value().squeeze(),
            'cash': pf_single_col.cash().squeeze(),
            'gross_exposure': pf_single_col.gross_exposure().squeeze(),
            'net_exposure': pf_single_col.net_exposure().squeeze()
        }, index=pf_single_col.wrapper.index)
        save_data(timeseries_df, 'portfolio_timeseries', strategy_dir)

        print("DEBUG: Saving drawdown data...")
        save_data(pf_single_col.drawdowns.records_readable, 'drawdowns', strategy_dir)

        print(f"\n--- Finished Simulation for: {strategy_name} ---")
        return stats

    except Exception as e:
        print(f"!!! AN ERROR OCCURRED during simulation of '{strategy_name}' on {ticker}: {e}")
        traceback.print_exc()
        return None


# Main
def main():
    """Main function to run simulations for all strategies and tickers."""
    try:
        with open(METADATA_FILE, 'r') as f:
            tickers = [item['Ticker'] for item in json.load(f)]
        tickers_to_process = tickers
    except FileNotFoundError:
        print(f"Error: Metadata file not found at {METADATA_FILE}")
        return

    results_dir = os.path.join(BASE_DIR, "simulation_results")
    os.makedirs(results_dir, exist_ok=True)
    
    all_results = []

    for ticker in tickers_to_process:
        print(f"\n{'#'*80}")
        print(f"##### Processing Ticker: {ticker} #####")
        print(f"{'#'*80}")

        df_baseline = load_simulation_data(ticker, 'Baseline')
        df_volume = load_simulation_data(ticker, 'Volume_Enhanced')
        df_dl = load_simulation_data(ticker, 'Deep_Learning_Enhanced')

        simulation_tasks = {
            "Baseline Breakout": (run_baseline_breakout, df_baseline, BEST_BASELINE_BREAKOUT_PARAMS),
            "Baseline Momentum": (run_baseline_momentum, df_baseline, BEST_BASELINE_MOMENTUM_PARAMS),
            "Baseline Bollinger Bands": (run_baseline_bbands, df_baseline, BEST_BASELINE_BBANDS_PARAMS),
            "Volume-Enhanced Breakout": (run_volume_breakout, df_volume, BEST_VOLUME_BREAKOUT_PARAMS),
            "Volume-Enhanced VWAP Reversion": (run_volume_vwap_reversion, df_volume, BEST_VOLUME_VWAP_REVERSION_PARAMS),
            "Volume-Enhanced Momentum": (run_volume_momentum, df_volume, BEST_VOLUME_MOMENTUM_PARAMS),
            "Deep Learning Breakout": (run_dl_breakout, df_dl, BEST_DL_BREAKOUT_PARAMS),
            "Deep Learning VWAP Reversion": (run_dl_vwap_reversion, df_dl, BEST_DL_VWAP_REVERSION_PARAMS),
            "Deep Learning Momentum": (run_dl_volume_momentum, df_dl, BEST_DL_VOLUME_MOMENTUM_PARAMS),
        }

        for name, (func, df, params) in simulation_tasks.items():
            stats = run_and_evaluate_strategy(name, ticker, func, df, params)
            if stats is not None:
                stats_df = stats.to_frame(name=ticker).T
                stats_df['Strategy'] = name
                stats_df['Ticker'] = ticker
                for p_name, p_val in params.items():
                    stats_df[p_name] = p_val[0]
                all_results.append(stats_df)

    if all_results:
        print("\n--- Aggregating all simulation results ---")
        final_results_df = pd.concat(all_results, ignore_index=True)
        
        all_param_keys = set()
        for d in [BEST_BASELINE_BREAKOUT_PARAMS, BEST_BASELINE_BBANDS_PARAMS, BEST_BASELINE_MOMENTUM_PARAMS,
                  BEST_VOLUME_BREAKOUT_PARAMS, BEST_VOLUME_MOMENTUM_PARAMS, BEST_VOLUME_VWAP_REVERSION_PARAMS,
                  BEST_DL_BREAKOUT_PARAMS, BEST_DL_VOLUME_MOMENTUM_PARAMS, BEST_DL_VWAP_REVERSION_PARAMS]:
            all_param_keys.update(d.keys())
        
        metric_cols = [col for col in final_results_df.columns if col not in ['Ticker', 'Strategy'] and col not in all_param_keys]
        param_cols = [col for col in final_results_df.columns if col in all_param_keys]
        
        final_cols = ['Ticker', 'Strategy'] + sorted(list(param_cols)) + metric_cols
        final_results_df = final_results_df.reindex(columns=final_cols)
        
        output_filename = os.path.join(results_dir, "simulation_summary.csv")
        final_results_df.to_csv(output_filename, index=False)
        print(f"\nSuccessfully saved comprehensive simulation summary to {output_filename}")
        print(final_results_df)
    else:
        print("\nNo successful simulations to save.")


In [ ]:
if __name__ == "__main__":
    main()